# Colecciones de imágenes en Google Earth Engine

En este cuaderno se van a estudiar los siguientes puntos relaccionados con el manejo de colecciones de imágenes:

*   Introducción.
*   Filtrado.
*   Información y metadatos.
*   Filtrado.
*   Mapping.
*   Reducción.
*   Composición y mosaicado.

Antes de empezar se va a preparar el entorno de trabajo identificando al usuario.
Posteiriormente se presenta la función para calcular el centro de una imagen y la representación de mapas vistos en cuadernos anteriores.


In [1]:
import ee

ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=e-whwKsMRRcph6xxz6lZHxcwOLIRPbliJImYVtyQXAk&tc=6CUqGpLhFnMKEdKDhwsrwTIvakvUMQxqoWBvJSqYU8w&cc=zzaW1HaHCCFukxakDGEeIjH9MKbTR5vKlkNtH0YrL00

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk43PUeBcIQBkD3Q-E4Ow_yajTPli7u_htwkZBGZWBC6QuK3yXkGdh8

Successfully saved authorization token.


In [2]:
def centro(imagen):
  coordenadas=imagen.get('system:footprint').getInfo()['coordinates']
  longitud=0
  latitud=0
  for i in range(len(coordenadas)):
    longitud= longitud+ coordenadas[i-1][0]
    latitud=latitud+coordenadas[i-1][1]
  
  longitud=longitud/(len(coordenadas))
  latitud=latitud/(len(coordenadas))
  return (latitud,longitud)

In [3]:
#Función para crear un mapa interactivo empleando el paquete folium
# Importamos libreria Folium
import folium

# Definicón de una función para representar imágenes Earth Engine en un folium map.
# 
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Añade un método de dibujo en folium.
folium.Map.add_ee_layer = add_ee_layer

### **1. Introducción a la colección de imágenes**

Hasta ahora se ha trabajado con imágenes individuales, necesitando conocer el identificador de la escena de satélite de interes, esto hace que el usuario tenga que realizar una tarea de exploración dentro de un dataset para localizar ésta, haciendo que en muchos casos este trabajo no sea eficiente.

Ahora se va a trabajar con una colección de imágenes almacenadas en uno de los catálogos o datasets dentro de GEE, permitiendo hacer búsqueda en base a:
*   una ventana temporal, 
*   un ámbito geográfico, 
*   que cumpla una determinada propiedad, etc.

###### **1.1 Acceso a colección de imágenes**

Para la creación de una variable de tipo colección de imágenes se empleará el tipo de dato **ee.ImageCollection**, indicando posteriormente el nombre del dataset de ímagenes con las que se quiere trabajar, así como todas aquellas condiciones que se estimen oportunas por parte del usuario.

    `newImageCollection = ee.ImageCollection( assetID)`

Siendo `assesID`  una cadena de texto con el identificador del catálogo de imágenes al que se desea acceder. En este momento uno se puede preguntar ¿qué catálogo de imágenes aparecen en GEE?, ¿cuál es su identificador?. Para ello, entrar en [https://earthengine.google.com](https://earthengine.google.com) y pulsar en [dataset](https://developers.google.com/earth-engine/datasets/).
 
Para cada uno de los dataset recogidos en GEE aparece una ficha descriptiva donde se encuentra el identificador para poder llamar mediante código a éste, concretamente hay que localizar el apartado *`Earth Engine Snippet`*.

Como ejemplo, el siguiente código crea una variable de tipo ImageCollection con todas las imágenes procedentes de Sentinel 2 en el dataset de reflectancia Top Of Atmosphere.

In [5]:
sentinel = ee.ImageCollection("COPERNICUS/S2").limit(100)

La variable sentinel será una variable de tipo ImageCollection que apuntará a todas las imágenes Sentinel 2 almacenadas en GEE. 

Si se imprime la información de la variable sentinel en la consola mediante **getInfo()**, ¿qué se obtiene?

In [6]:
sentinel.getInfo()

{'type': 'ImageCollection',
 'bands': [],
 'id': 'COPERNICUS/S2',
 'version': 1680445006332685,
 'properties': {'date_range': [1435017600000, 1647993600000],
  'period': 0,
  'system:visualization_0_min': '0.0',
  'type_name': 'ImageCollection',
  'keywords': ['copernicus', 'esa', 'eu', 'msi', 'radiance', 'sentinel'],
  'system:visualization_0_bands': 'B4,B3,B2',
  'thumb': 'https://mw1.google.com/ges/dd/images/sentinel2_thumb.png',
  'description': '<p>Sentinel-2 is a wide-swath, high-resolution, multi-spectral\nimaging mission supporting Copernicus Land Monitoring studies,\nincluding the monitoring of vegetation, soil and water cover,\nas well as observation of inland waterways and coastal areas.</p><p>The Sentinel-2 data contain 13 UINT16 spectral bands representing\nTOA reflectance scaled by 10000. See the <a href="https://sentinel.esa.int/documents/247904/685211/Sentinel-2_User_Handbook">Sentinel-2 User Handbook</a>\nfor details. In addition, three QA bands are present where one\n

Efectivamente, se ha obtenido un error debido a que GEE esta limitado para imprimir y mostrar resultados como mucho de **5000 elementos**. Para colecciones mayores a 5000 imágenes será necesario filtrar las imágenes que sean de interés antes de imprimir la colección. 

De este modo, una variable objeto tipo ImageCollection en GEE se corresponde con un conjunto de imágenes que representan variaciones espaciales considerando aquellas:
* Para un mismo lugar y distintas fechas.
* Para diferentes lugares en el mismo instante de tiempo.
* Para diferentes lugares y diferentes momentos.

Además, como se verá ma adelante, una ImageCollection se puede procesar mediante el uso de distintas operaciones para limitar el número de imágenes, filtrado, transformación, representación, etc. En este cuaderno se van a ver algunas de ellas.

## **2. Filtrado de ee.imageCollection**

Como ya se ha adelantado, GEE presenta un conjunto de métodos que podemos emplear según nuestros intereses para filtrar una colección de imágenes. Estos filtros permitirán seleccionar el conjunto de imágenes dentro de un `dataset`de interés, no siendo necesario cargar todas las escenas incluidas en él. Los tipos de filtrados que se pueden hacer son:
* **imageCollection.limit** limita el número de imágenes de la colección a un número definido por el usuario.
* **imageCollection.filterDate()** filtra considerando una ventana temporal.
* **imageCollection.filterBounds()** filtra empleando un entorno geográfico.
* **imageCollection.filterMetadata()** filtra por una propiedad de los metadatos.

Estos métodos se pueden combinar con objeto de acotar la colección a los intereses del desarrollador.

### **2.1 imagenColeecion.limit()**
Crea una nueva colección de imágenes que contiene solamente los *n*  elementos definidos por el usuario de otra colección de imágenes específica. 
Con carácter opcional posteriormente es posible ordenar estos elementos en orden creciente o decreciente según el valor de alguna de sus propiedades.

    `coleccion = oldImageCollection.limit(num,propiedad, orden)`

* `coleccion`: nueva colección de imágenes.
* `oldImageCollection`: una colección de imágenes específica.
*`num`: numero de elementos, debe ser un valor entero.
* `propiedad` (opcional): se especifica que propiedad se va a considerar. Tipo de dato string.
*`orden`: Ascendente (True) o descendente (False).

El siguiente script crea una colección de imágenes con las 100 primeras escenas dentro del dataset COPERNICUS/S2. Posteriormente, mediante el método *getInfo* se imprimen los metadatos de todas y cada una de las escenas que componen la colección.

In [7]:
# Este script crea una primera colección con todas las imágenes Sentinel 2 y posteriormente crea otra limitándolas a 100.
coleccion=ee.ImageCollection('COPERNICUS/S2')
coleccion2=coleccion.limit(100)

coleccion2.getInfo() #Imprime la información de los elementos de la colección

{'type': 'ImageCollection',
 'bands': [],
 'id': 'COPERNICUS/S2',
 'version': 1680445006332685,
 'properties': {'date_range': [1435017600000, 1647993600000],
  'period': 0,
  'system:visualization_0_min': '0.0',
  'type_name': 'ImageCollection',
  'keywords': ['copernicus', 'esa', 'eu', 'msi', 'radiance', 'sentinel'],
  'system:visualization_0_bands': 'B4,B3,B2',
  'thumb': 'https://mw1.google.com/ges/dd/images/sentinel2_thumb.png',
  'description': '<p>Sentinel-2 is a wide-swath, high-resolution, multi-spectral\nimaging mission supporting Copernicus Land Monitoring studies,\nincluding the monitoring of vegetation, soil and water cover,\nas well as observation of inland waterways and coastal areas.</p><p>The Sentinel-2 data contain 13 UINT16 spectral bands representing\nTOA reflectance scaled by 10000. See the <a href="https://sentinel.esa.int/documents/247904/685211/Sentinel-2_User_Handbook">Sentinel-2 User Handbook</a>\nfor details. In addition, three QA bands are present where one\n

In [8]:
# Este script hace lo mismo que el anterior pero encadena el método limit
coleccion2=ee.ImageCollection('COPERNICUS/S2').limit(100)
coleccion2.getInfo()

coleccion2.getInfo() #Imprime la información de los elementos de la colección

{'type': 'ImageCollection',
 'bands': [],
 'id': 'COPERNICUS/S2',
 'version': 1680445006332685,
 'properties': {'date_range': [1435017600000, 1647993600000],
  'period': 0,
  'system:visualization_0_min': '0.0',
  'type_name': 'ImageCollection',
  'keywords': ['copernicus', 'esa', 'eu', 'msi', 'radiance', 'sentinel'],
  'system:visualization_0_bands': 'B4,B3,B2',
  'thumb': 'https://mw1.google.com/ges/dd/images/sentinel2_thumb.png',
  'description': '<p>Sentinel-2 is a wide-swath, high-resolution, multi-spectral\nimaging mission supporting Copernicus Land Monitoring studies,\nincluding the monitoring of vegetation, soil and water cover,\nas well as observation of inland waterways and coastal areas.</p><p>The Sentinel-2 data contain 13 UINT16 spectral bands representing\nTOA reflectance scaled by 10000. See the <a href="https://sentinel.esa.int/documents/247904/685211/Sentinel-2_User_Handbook">Sentinel-2 User Handbook</a>\nfor details. In addition, three QA bands are present where one\n

Ya se ha visto que una colección de imágenes se compone de un conjunto de imágenes, donde en ocasiones puede ser necesario conocer datos particulares de cada una de ellas. 

En el siguiente script se presenta como, por ejemplo se puede conocer el identificador de cada una de las imágenes que componen una colección de imágenes. Para ello, en primer lugar es necesario convertir la colección en una lista, empleando para ello el método **toList()**. El siguiente paso es recorrer la lista mediante un bucle **for**, creando una variable denominada *imagen* sobre la cual, a través del método *getInfo()* se esta accediendo al elemento de los metadatos de la escena **id**, el cual contiene el identificador de la escena.

In [9]:
coleccion=ee.ImageCollection('COPERNICUS/S2').limit(10)

lista=coleccion.toList(coleccion.size())

for i in range(lista.size().getInfo()):
  imagen=lista.get(i)
  print  (imagen.getInfo()['id'])

COPERNICUS/S2/20150627T102531_20160606T223605_T31RCL
COPERNICUS/S2/20150627T102531_20160606T223605_T31RCM
COPERNICUS/S2/20150627T102531_20160606T223605_T31RDL
COPERNICUS/S2/20150627T102531_20160606T223605_T31RDM
COPERNICUS/S2/20150627T102531_20160606T223605_T31RDN
COPERNICUS/S2/20150627T102531_20160606T223605_T31RDP
COPERNICUS/S2/20150627T102531_20160606T223605_T31RDQ
COPERNICUS/S2/20150627T102531_20160606T223605_T31REL
COPERNICUS/S2/20150627T102531_20160606T223605_T31REM
COPERNICUS/S2/20150627T102531_20160606T223605_T31REN


Con el script anterior, en cada paso por el bucle se esta extrayendo el nombre del repositorio y el identificador de las escenas que componen la colección. De este modo, se podría aplicar todo lo estudiado en el cuaderno anterior sobre imágenes, creando así una variable de tipo *ee.Image*.

En el siguiente ejemplo lo que se hace es calcular las coordenadas del punto central de una colección de imágenes, para ello será necesario conocer cada uno de las coordenadas centrales de las imágenes que componen una colección de imagenes. 
Así, se recorre en un primer bucle *for* la lista de imágenes. Dentro de este bucle se van extrayendo las coordenadas recogidas en una lista de cada una de las imágenes. Esta información aparece en el elemento **coordinates** dentro de **system:footprint**. Conforme se van extrayendo las imágenes de la colección se recorre la lista de coordenadas con un segundo bucle para calcular las coordenadas centrales de cada una de las imágenes. Estas coordenadas se van sumando, almacenando esta suma en las variables `latitud_suma` y `longitud_suma`, que finalmente serán divididos por el total de imágenes de la colección para obtener las coordenadas centrales de la zona geográfica cubierta por la colección.

In [10]:
coleccion=ee.ImageCollection('COPERNICUS/S2').limit(10)

lista=coleccion.toList(coleccion.size())
latitud_suma=0
longitud_suma=0
for i in range(0,lista.size().getInfo()):
  imagen=ee.Image(lista.get(i))
  coordenadas=imagen.get('system:footprint').getInfo()['coordinates']
  longitud=0
  latitud=0
  for j in range(len(coordenadas)):
    longitud= longitud+coordenadas[j-1][0]
    latitud=latitud+coordenadas[j-1][1]

  longitud=longitud/(len(coordenadas))
  latitud=latitud/(len(coordenadas))

  latitud_suma=latitud_suma+latitud
  longitud_suma=longitud_suma+longitud

latitud_central=latitud_suma/lista.size().getInfo()
longitud_central=longitud_suma/lista.size().getInfo()

print ('latitud central: ', latitud_central)
print ('longitud central: ',longitud_central)

latitud central:  28.86622135958214
longitud central:  2.701522682793853


Con objeto de poder reutilizar este código se ha creado una función denominada **centro_coleccion**, presentando como parámetro de entrada una variable que tiene que ser de tipo *ee.ImageCollection*, sienod la salida igual a la latitud y longitud central.

In [11]:
#Función para calcular las coordenadas centrales de una colección de imágenes.

def centro_coleccion(coleccion):
  lista=coleccion.toList(coleccion.size())
  latitud_suma=0
  longitud_suma=0
  for i in range(0,lista.size().getInfo()):
    imagen=ee.Image(lista.get(i))
    coordenadas=imagen.get('system:footprint').getInfo()['coordinates']
    longitud=0
    latitud=0
    for j in range(len(coordenadas)):
      longitud= longitud+ coordenadas[j-1][0]
      latitud=latitud+coordenadas[j-1][1]

      longitud=longitud/(len(coordenadas))
      latitud=latitud/(len(coordenadas))

      latitud_suma=latitud_suma+latitud
      longitud_suma=longitud_suma+longitud

  latitud_central=latitud_suma/lista.size().getInfo()
  longitud_central=longitud_suma/lista.size().getInfo()

  return (latitud_central,longitud_central)



Antes de continuar con el filtrado de colecciones de imágenes se va a estudiar como poder visualizar en pantalla una colección de imágenes. En el cuaderno anterior se imprimía en pantalla una imagen individual. En este caso, al tener una colección de imágenes compuesta de *n* imágenes el procedimiento no puede ser el mismo. Para visualizar en pantalla una colección de imágenes, se deberá generar una **imagen sintética** resumen del  conjunto de imágenes que componen una colección, siendo esta la que se imprimirá en pantalla.

En el siguiente código se facilita al alumno la función **Mapdisplay** para representar una colección de imágenes. Las entradas de esta función son:

* center: tupla con las coordenadas del centro de la colección de imágenes.
* dicc: diccionario con geometrías o imágenes a representar.
* Tiles: fondo del mapa a emplear.
* zoom_start: nivel de zoom.

In [12]:
import folium

# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

En el siguiente ejemplo se imprimen en pantalla las primeras 100 escenas del repositorio COPERNICUS/S2.

In [13]:
#Creación de una colección de imágenes Sentinel-2 con las 100 primeras imágenes del dataset
coleccion=ee.ImageCollection('COPERNICUS/S2').limit(100)

#Definición de los parámetros de visualización
vis_params = {
  'min': 0,
  'max': 6000,
  'bands': ['B4', 'B2', 'B3']
}

coordenadas = centro_coleccion(coleccion)
                 
dicc = {
    'Colección Sentinel' : coleccion.getMapId(vis_params),
}

# Display the results
Mapdisplay(coordenadas, dicc, zoom_start= 4)

### **2.2 imageCollection.filterMetadata()**
Permite crear una nueva colección de imágenes que contiene solamente aquellas imágenes de una colección cuyo valor para una propiedad específica cumple una determinada condición.

> `coleccion = oldImageCollection.filterMetadata(propiedad, condicion, valor)`

Siendo:
* `propiedad`: cadena de texto con la propiedad específica.
* `condición`: "equals", "less_than", "greater_than""starts_with", "ends_with", "contains", "not_equals", "not_less_than", "not_greater_than", "not_starts_with", "not_ends_with"o "not_contains".
* `valor`: un valor específico a comparar, pudiendo ser un número o una cadena de texto.

En el siguiente código  se filtra, en primer lugar, aquellas imágenes que presenta un valor inferior a 0.1 en la propiedad `'CLOUD_COVERAGE_ASSESSMENT'` (cobertura nubosa) dentro del repositorio de Copernicus/S2_SR y finalmente se limita el número de imágenes a un total de 100 imágenes.

In [14]:
coleccion=ee.ImageCollection('COPERNICUS/S2_SR');
coleccion2 = coleccion.filterMetadata('CLOUD_COVERAGE_ASSESSMENT' ,'less_than',0.1).limit(100);

#Obtenemos el centro de la colección de imágenes.
coordenadas=centro_coleccion(coleccion2) #Se esta llamando a la función que se ha creado anteriormente.  

vis_params = {
  'min': 0,
  'max': 3500,
  'bands': ['B4', 'B2', 'B3']
}

dicc = {
    'coleccion' : coleccion2.getMapId(vis_params)

}

# Display the results
Mapdisplay(coordenadas, dicc, zoom_start= 3)

### **2.3 imagenCollection.filterDate**
Crea una colección de imágenes conteniendo solo aquellas imágenes que se encuentran dentro de una ventana temporal definida por el usuario.

> `newImageCollection = oldImageCollection.filterDate( f_inicio, f_fin)`

El formato de la fecha de inicio y fin puede ser año/mes/día o año-mes-día o bien el número de milisegundos desde el 1 de enero de 1970.

En el sigueinte ejemplo se filtra la ImageCollection en primer lugar siguiendo un criterio temporal, devolviendo todas las imágenes Sentinel 2 con reflectancia a nivel de suelo entre el 1 de enero y el 31 de diciembre de 2018. Del conjunto de imágenes dentro de esa ventana temporal se filtran posteriormente solo aquellas escenas con una cobertura nubosa inferior a 0.1, limitando finalmente la colección a las primeras 100 escenas.


In [15]:
coleccion=ee.ImageCollection('COPERNICUS/S2_SR')

coleccion2 = coleccion.filterDate('2018-01-01','2018-12-31')\
                      .filterMetadata('CLOUD_COVERAGE_ASSESSMENT' ,'less_than',0.1)\
                      .limit(100)

#Definición de los parámetros de visualización
vis_params = {
  'min': 0,
  'max': 3500,
  'bands': ['B4', 'B2', 'B3']
}
#Obtenemos el centro de la colección de imágenes.
coordenadas=centro_coleccion(coleccion2)   

dicc = {
    'coleccion' : coleccion2.getMapId(vis_params)
}

# Obtenemos el número de imágenes
num_imagenes = coleccion2.size()
print('Número de imágenes: ', num_imagenes.getInfo())

# Display the results
Mapdisplay(coordenadas, dicc, zoom_start= 3)

Número de imágenes:  100


### **2.4 imageCollection.filterBounds()**

Crea una colección con aquellas imágenes que intersectan con una geometría, entidad o colección de entidades definida.

> `newImageCollection = oldImageCollection.filterBounds(geometria)`

Mas adelante se estudiará como trabajar con geometrías en profundidad.

Mediante **ee.Geometry.Point** se ha creado una geometría de tipo punto indicando longitud y latitud. Posteriormente esta geometría es empleada por el método **filterBounds()** para seleccionar aquellas escenas que intersectan con esta geometría. 
En este ejemplo, de todas esas escenas se seleccionan aquellas registradas en el año 2017 y que además presentan una cobertura nubosa inferior a 0.1.

In [16]:
coleccion=ee.ImageCollection('COPERNICUS/S2')

punto = ee.Geometry.Point(-4.71,37.93) #Definición de un punto

coleccion2 = coleccion.filterBounds(punto)\
      .filterDate('2017-01-01','2017-12-31')\
      .filterMetadata('CLOUD_COVERAGE_ASSESSMENT' ,'less_than',0.1)

vis_params = {
  'min': 500,
  'max': 2500,
  'bands': ['B4', 'B2', 'B3'],
  'gamma':2
}

dicc = {
     'coleccion' : coleccion2.getMapId(vis_params)
}

coord_punto = punto.getInfo()['coordinates'] #Creamos esta variable para obtener las coordenadas a partir de la geometría creada anteriormente 
coord_punto.reverse()
print (coord_punto)
# Obtenemos el número de imágenes
num_imagenes = coleccion2.size()
print('Número de imágenes: ', num_imagenes.getInfo())

# Display the results
Mapdisplay(coord_punto, dicc, zoom_start= 10)

[37.93, -4.71]
Número de imágenes:  90


#### **2.4.1 Cargar información geográfica desde Google Drive**
En el ejemplo anterior se ha definido mediante un **ee.Geometry.Point** la geometría para filtrar la colección de imágenes. 

Ahora se va a aprender como emplear geometrías almacenadas en un fichero externo como elemento de filtrado espacial. En este ejemplo se tiene almacenada en el espacio de Google Drive del usuario una tabla denominada *parcelas* con la geometría de varias parcelas que vamos a emplear para filtrar una colección de imágenes, entendiendo esta tabla como un fichero shapefile.

Para ello deberemos aprender como acceder a un fichero con dichas geometrías. Los requisitos a considerar son:
1. El formato de archivo será shapefile.
2. Deberemos tenerlo subido a Google Drive.

En primer lugar, se tendrá que tener instaladas las librerias **geetools** y **pycrs** en la máquina virtual. Como esta máquina es dinámica se deberán instalar las librerías cada vez que se inicie una sesión.

En la siguiente celda de código se muestra como instalar estas librerias (El signo ! delante de **pip** permite simular una terminal de comandos de Python.

In [17]:
!pip install geetools #Exclamación para simular terminal de comandos. Esta libreria contiene muchas herramientas para GEE
#!pip install -U geetools
!pip install pycrs #Dependencia de la anterior
#!pip install fastkml
!pip install ipygee

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 KB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 KB 4.8 MB/s eta 0:00:00
  Created wheel for geetools: filename=geetools-0.6.14-py3-none-any.whl size=92107 sha256=3b004f5f000c3da4384184ac9f9a62125230086a91f818d7a366c0c67a7cd923
  Stored in directory: /root/.cache/pip/wheels/20/4c/bf/0228d788d820ac3cf05cbaa753965eb6396b5d3fe11ed912e5
Successfully built geetools
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pycrs: filename=PyCRS-1.0.2-py3-none-any.whl size=32702 sha256=07b1244df1af730aba2c8099988d97ff434d426c7ee1570c35ecc5ab746f0c75
  Stored in directory: /root/.cache/pip/wheels/94/01/24/bc7bff66667ef317615144a15e04593a08d9bb322f2c427d6c
Successfully bu

In [ ]:
# Permite cargar shapefiles y convertirlos en una featureCollectionLoad 
!pip install geetools
!pip install fastkml
!pip install pycrs
!pip install geemap

# Permite mostrar series como las vemos en Earth Engine Code Editor 
!pip install ipygee

In [ ]:
!pip install geemap

El siguiente paso es, mediante código, poder acceder a nuestro espacio en **Google Drive**. Para ello cargaremos la libreria **google.colab** la cual permitirá identificarnos para poder acceder a nuestra unidad.
En la siguiente celda se presenta el código para este fin, el proceso de autenticarse se hace en dos fases.


In [18]:
#Acceso a nuestro espacio en Google Drive
from google.colab import auth, drive
auth.authenticate_user()

drive.mount('/content/drive')

Mounted at /content/drive


Una vez cargada la unidad de Google Drive del usuario se emplearán las funciones de la libreria `geetools` para, una vez importado al cuaderno `batch`, poder acceder al archivo shape de interes. En el ejemplo presentado mas abajo se ha definido una variable denominada `path_file` que contienen la ruta y el nombre del archivo shapefile al que se quiere acceder. Posteriormente, la variable `parcela` lee el contenido de éste.




En el siguiente código se crea una variable denominada `parcela`  con las geometrías almacenadas en dicha tabla, siendo esta variable empleada para filtrar espacialmente aquellas imágenes de Sentinel 2 con reflectancia a nivel de superficie entre las fechas 1-1-2020 y 14-02-2020 y con un cobertura de nubes inferior al 1%.

Con objeto de centrar el mapa a generar sobre las parcelas de interes se obtiene el centroide del conjunto de geometrías. Para ello, sobre el método **geometry()** aplicamos el método **centroid()**. Como el orden de las coordenadas no coincide con el que empleamos en la función para pintar a cartografía invertimos el orden de etas mediante el método **reverse()**.

In [20]:
!pip install geetools
!pip install fastkml
!pip install pycrs
!pip install geemap

# Permite mostrar series como las vemos en Earth Engine Code Editor 
!pip install ipygee

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pygeoif: filename=pygeoif-0.7-py3-none-any.whl size=19015 sha256=96480a0a2b867a6d0e268272808fcfe45806931064e3babdd28f68224aada504
  Stored in directory: /root/.cache/pip/wheels/1f/03/f4/8041459258b17a1bc9c057a947cc1841a21084b8c8f06877c3
Successfully built pygeoif
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 39.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.5 MB/s eta 0:00:00


In [82]:
import os
import geemap
from geetools import batch

ee.Initialize()

#Lectura en formato shape
path_file = os.path.join('/content/drive/MyDrive/Colab Notebooks/HIBA/Cuadernos Colab/3 - Colecciones de imagenes/datos/parcela_demo.shp')
parcela = geemap.shp_to_ee(path_file)

coleccion=ee.ImageCollection('COPERNICUS/S2_SR')

#punto = ee.Geometry.Point(-4.71,37.93) #Definición de un punto
coleccion2 = coleccion.filterBounds(parcela)\
      .filterDate('2020-01-01','2020-02-14')\
      .filterMetadata('CLOUD_COVERAGE_ASSESSMENT' ,'less_than',1)\

# Obtenemos el número de imágenes
num_imagenes = coleccion2.size()
print('Número de imágenes: ', num_imagenes.getInfo())

#Definición de los parámetros de visualización
vis_params = {
  'min': 500,
  'max': 2500,
  'bands': ['B8', 'B3', 'B4'],
  'gamma':2
}

#Determinación del centroide de las geometrías contenidas en la tabla
poligono=ee.Geometry.Polygon(parcela.geometry().getInfo()['coordinates'])
centroide = poligono.centroid().getInfo()['coordinates']
centroide.reverse() #invertimos el orden de las coordenadas para que coincida con el orden de la función de representación

#Creamos un diccionario con la colección de imágenes y las parcelas, que pasaremos como parámetro para pintar el mapa.
dicc = {
     'coleccion' : coleccion2.getMapId(vis_params),
     'parcela':parcela.getInfo()
}

# Display the results
Mapdisplay(centroide, dicc, zoom_start= 17)

Número de imágenes:  2


### **2.5 imageCollection.select**
Crea una nueva colección de imágenes conteniendo sola y exclusivamente un conjunto determinado de bandas seleccionadas de otra colección de imágenes.

`newImageCollection = oldImageCollection.select(oldBands, newBandNames)`
  * `oldBands`: Los nombres de las bandas espectrales a considerar pasadas en modo array.
  * `newBandNames`(opcional): Array con los nombres nuevos a asignar a las bandas seleccionadas.

  En este ejemplo se ha incluido el método select seleccionando solamente las bandas B8 y B5, renombrándolas además como “NIR” y “RED” 


In [83]:
coleccion = ee.ImageCollection('COPERNICUS/S2')\
              .filterBounds(parcela)\
              .select(['B8','B5'],['NIR','RED']);

## **3. Transformación de colecciones de imágenes**



### **3.1 imageCollection.mosaic**

Crea una imagen única por combinación de distintas imágenes solapadas pertenecientes a una colección considerando el orden dentro de la colección, es decir, pondrá arriba las que aparezcan al final de la colección.

`newImage = oldImageCollection.mosaic()`

En el ejemplo siguiente se ha creado una imagen (variable *sentinel*) a partir de la selección dentro del dataset de Sentinel 2 SR de las imágenes que intersectan con la(s) geometría(s) que contiene la tabla dinámica *parcela*, donde la imagen contiene solo las bandas B8, B4 y B3, donde al final se llama al método **mosaic()**.

Sobre este mosaico a la hora de crear el diccionario para posteriormente crear el visor se puede comprobar como se ha empleado el método **clip()**, pasando como parámetro la FeatureCollection `parcela`. Esto permite recortar el mosaico por las geometrías consideradas.

In [87]:
#Leemos un shapeFile alojado en nuetro espacio en Google Drive
path_file = os.path.join('/content/drive/MyDrive/Colab Notebooks/HIBA/Cuadernos Colab/3 - Colecciones de imagenes/datos/parcela_demo.shp')
parcela = geemap.shp_to_ee(path_file)

sentinel = ee.ImageCollection('COPERNICUS/S2')\
              .filterBounds(parcela)\
              .select(['B8','B4','B3'])\
              .filterDate('2019-01-01','2019-02-14')\

mosaico=sentinel.mosaic()

poligono=ee.Geometry.Polygon(parcela.geometry().getInfo()['coordinates'])
centroide = poligono.centroid().getInfo()['coordinates']
centroide.reverse()

vis_params = {
  'min': 500,
  'max': 2500,
  'bands': ['B8', 'B3', 'B4'],
}
dicc = {
     'coleccion' : sentinel.getMapId(vis_params),
     'mosaico' : mosaico.clip(parcela).getMapId(vis_params),
     'parcela':parcela.getInfo()
}

# Display the results
Mapdisplay(centroide, dicc, zoom_start= 15)

## **4 Reducción de una colección de imágenes**

Un reductor es un mecanismo para agregar datos bien sea a lo largo del tiempo, el espacio, bandas espectrales, etc. Para ello se empleará la clase **ee.Reducer**, indicando como se desea agregar.

Para componer una imágen a partir de una imageCollection se emplearemos

` imageCollectin.*reduce()*`

El resultado será una composión de todas las imágenes de una colección en una imagen individual sintética. 

![texto alternativo](https://developers.google.com/earth-engine/images/Reduce_ImageCollection.png)





Los métodos de reducción que podemos emplear son:
 
`imageCollection.sum, .product, .max, min, .mean, .mode ,.median, and .count`

Crea una nueva imagen en la que el valor de cada píxel se calcula como una función matemática de los valores de cada píxel considerando las bandas correspondientes de las imágenes en una colección de imágenes específica.

`newImage = oldImageCollection.sum( ) [o .product( ) o .max( ) o .min( ) o .mean( ) o .mode() o .median() o .and() o .count()]`

En el siguiente ejemplo, para cada posición fila/columna de cada banda de la imagen de salida, el valor será el valor medio de los pixeles contenidos en las imágenes de la colección.

In [90]:
#Leemos un shapeFile alojado en nuetro espacio en Google Drive
path_file = os.path.join('/content/drive/MyDrive/Colab Notebooks/HIBA/Cuadernos Colab/3 - Colecciones de imagenes/datos/parcela_demo.shp')
parcela = geemap.shp_to_ee(path_file)

sentinel = ee.ImageCollection('COPERNICUS/S2')\
              .filterDate('2018-09-01','2019-08-31')\
              .filterBounds(parcela)\
              .filterMetadata('CLOUD_COVERAGE_ASSESSMENT' ,'less_than',0.1)\
              .select(['B8','B4','B3'])

imagen_media = sentinel.mean()
imagen_maximo = sentinel.max()

poligono=ee.Geometry.Polygon(parcela.geometry().getInfo()['coordinates'])
centroide = poligono.centroid().getInfo()['coordinates']
centroide.reverse()

vis_params = {
  'min': 0,
  'max': 5000,
  'bands': ['B8', 'B3', 'B4'],
}
dicc = {
     'Media' : imagen_media.getMapId(vis_params),
     'Máximo' : imagen_maximo.getMapId(vis_params),
     'parcela':parcela.getInfo()
}

# Display the results
Mapdisplay(centroide, dicc, zoom_start= 16)

### **4.1 Estadística sobre una imagen de una geometría**
Para obtener la estadística de los valores de los píxeles sobre una imagen **ee.Image** se empleará **image.reduceRegion()**. Lo que obtendremos es la estadística de los pixeles bajo la región de interés. Esta región puede ser bien un polígono o bien un punto.

![texto alternativo](https://developers.google.com/earth-engine/images/Reduce_region_diagram.png)

In [91]:
#Leemos un shapeFile alojado en nuetro espacio en Google Drive
path_file = os.path.join('/content/drive/MyDrive/Colab Notebooks/HIBA/Cuadernos Colab/3 - Colecciones de imagenes/datos/parcela_demo.shp')
parcela = geemap.shp_to_ee(path_file)

parcela = ee.Feature(parcela.first()).geometry() #Tomamos la geometría de la primera feature

imagen = ee.ImageCollection('COPERNICUS/S2')\
              .filterDate('2018-09-01','2019-08-31')\
              .filterBounds(parcela)\
              .filterMetadata('CLOUD_COVERAGE_ASSESSMENT' ,'less_than',0.1).first()

#Obtenemos el valor medio de todas las bandas de la imagen teniendo en cuenta la geometría de la primera entidad
meanDiccionario = imagen.reduceRegion(**{
  'reducer': ee.Reducer.mean(),
  'geometry': parcela,
  'scale': 10,
  'maxPixels': 1e9
});

#The result is a Dictionary.  Print it.
meanDiccionario.getInfo()

{'B1': 1493.0589471539017,
 'B10': 12.720777980155653,
 'B11': 2539.833914992805,
 'B12': 1414.7654914597958,
 'B2': 1252.4581014125408,
 'B3': 1297.954840722956,
 'B4': 1202.0639146106912,
 'B5': 1619.2216377322914,
 'B6': 2915.6144234565845,
 'B7': 3367.7848326986023,
 'B8': 3165.7948440345926,
 'B8A': 3645.2625746710623,
 'B9': 669.2051814395433,
 'QA10': 0,
 'QA20': 0,
 'QA60': 0}

### **4.2 Estadística sobre una imagen de un conjunto de geometrías**
Para obtener la estadística de los valores de los píxeles sobre una imagen **ee.Image** de varias geometrías se empleará **image.reduceRegions()**. Lo que obtendremos es la estadística de los pixeles bajo cada una de las regiones de interés.

In [92]:
#Leemos un shapeFile alojado en nuetro espacio en Google Drive
path_file = os.path.join('/content/drive/MyDrive/Colab Notebooks/HIBA/Cuadernos Colab/3 - Colecciones de imagenes/datos/parcela_demo.shp') 
parcela = geemap.shp_to_ee(path_file)

# Load input imagery: Landsat 7 5-year composite.
imagen = ee.ImageCollection('COPERNICUS/S2')\
              .filterDate('2018-09-01','2019-08-31')\
              .filterBounds(parcela)\
              .filterMetadata('CLOUD_COVERAGE_ASSESSMENT' ,'less_than',0.1).first()

# Add reducer output to the Features in the collection.
resumen = imagen.reduceRegions(**{
  'collection': parcela,
  'reducer': ee.Reducer.mean(),
  'scale': 10,
})

resumen.getInfo()

{'type': 'FeatureCollection',
 'columns': {'B1': 'Float<0.0, 65535.0>',
  'B10': 'Float<0.0, 65535.0>',
  'B11': 'Float<0.0, 65535.0>',
  'B12': 'Float<0.0, 65535.0>',
  'B2': 'Float<0.0, 65535.0>',
  'B3': 'Float<0.0, 65535.0>',
  'B4': 'Float<0.0, 65535.0>',
  'B5': 'Float<0.0, 65535.0>',
  'B6': 'Float<0.0, 65535.0>',
  'B7': 'Float<0.0, 65535.0>',
  'B8': 'Float<0.0, 65535.0>',
  'B8A': 'Float<0.0, 65535.0>',
  'B9': 'Float<0.0, 65535.0>',
  'QA10': 'Float<0.0, 65535.0>',
  'QA20': 'Float<0.0, 4.294967295E9>',
  'QA60': 'Float<0.0, 65535.0>',
  'system:index': 'String'},
 'features': [{'type': 'Feature',
   'geometry': {'geodesic': False,
    'type': 'Polygon',
    'coordinates': [[[-5.139542717229896, 37.79174345705338],
      [-5.139472141479828, 37.7915277023614],
      [-5.139280935407523, 37.7893608067002],
      [-5.142219561309627, 37.78861797832703],
      [-5.142256509954761, 37.788783275594554],
      [-5.141776255263413, 37.78997932990016],
      [-5.141891658583806, 37.

## **5. Mapping sobre una colección de imágenes**


En el cuaderno anterior se estudió como calcular el índice NDVI para una imagen individual. En este cuaderno estamos trabajando con colecciones de imágenes, el objetivo ahora es saber como poder calcular por ejemplo el índice NDVI para cada una de las imagenes que componen nuestra colección.


Para ello usaremos imageCollection.**map()**, permitiendo aplicar una función para cada una de las imágenes individuales de una colección de imágenes. El único argumento que se pasa a `map()` es el nombre de una función. Como resultado se puede tener:
1. Una nueva colección, resultante de aplicar una función específica a cada una de las imágenes.
2. Incorporar una nueva banda a las ya existente dentro de cada una de las imágenes.
3. Un dato resultante de la aplicación de la función.

En el caso de querer crear una nueva colección de imágenes se escribirá:

`Nueva_coleccion_ = colección_original.map ( *nombre_función* )`

Siendo *nombre_función* el nombre de la función que se quiere pasar a la colección.

En este ejemplo se ha creado una función que se llama NDVI para calcular el valor de este índice sobre una imagen. Se ha cargado la información geométrica de un shapefile que se encuentra en nuestro espacio de Google Drive. Posteriormente se crea una colección de imágenes de Sentinel 2 con valores de reflectancia a suelo con todas las imágenes que intersectan con las geometrías almacenadas en el shapefile entre dos fechas definidas y que presentan un porcentaje de nubes inferior al 5%.

A continuación se crea la colección de imágenes *coleccion_ndvi* la cual tiene el mismo número de imágenes, teniendo cada imagen una sola banda correspondiente con el cálculo de NDVI.

Finalmente, se pinta en pantalla alguna información de ambas colecciones.


In [93]:
def NDVI(entrada):
  ndvi = entrada.normalizedDifference(['B8','B4'])
  return (ndvi)


coleccion_sentinel = ee.ImageCollection('COPERNICUS/S2_SR')\
           .filterBounds(parcela)\
           .filterDate('2018-01-01','2018-06-30')\
           .filterMetadata('CLOUD_COVERAGE_ASSESSMENT','less_than',5)

coleccion_ndvi=coleccion_sentinel.map(NDVI)

num_imagenes = coleccion_sentinel.size()
print('Número de imágenes colección Sentinel 2: ', num_imagenes.getInfo())
print('Número de imágenes colección NDVI: ', coleccion_ndvi.size().getInfo())
print ()
print('Información de una de las imágenes de la colección de NDVI')
imagen = ee.Image(coleccion_ndvi.first());

imagen_2=ee.Image(coleccion_sentinel.first());
imagen.getInfo()


Número de imágenes colección Sentinel 2:  17
Número de imágenes colección NDVI:  17

Información de una de las imágenes de la colección de NDVI


{'type': 'Image',
 'bands': [{'id': 'nd',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': -1,
    'max': 1},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32630',
   'crs_transform': [10, 0, 300000, 0, -10, 4200000]}],
 'properties': {'system:footprint': {'type': 'LinearRing',
   'coordinates': [[-5.245889968659827, 36.93681991133494],
    [-5.245800249632771, 36.936731609676],
    [-4.564182464657918, 36.947668766198206],
    [-4.563814108442841, 36.94807674630171],
    [-4.563480071341085, 36.94863569440024],
    [-4.558369340331491, 36.962500596520826],
    [-4.550070703889545, 36.985324538324505],
    [-4.532262846985197, 37.038556256983995],
    [-4.439049751711759, 37.31742604796306],
    [-4.281810836810058, 37.78457543233459],
    [-4.259751002494333, 37.84970434024901],
    [-4.231149285897886, 37.93409442370001],
    [-4.229199926916339, 37.940064627660654],
    [-4.2291967200212754, 37.940087020348315],
    [-4.229214709922922, 37.94116836855305


Si observamos en la información que hemos obtenido en la colección anterior, se ha perdido toda la información contenida en los metadatos y quizas, en función de la aplicación a desarrollar, sea necesario tenerlos.

Para ello, en el siguiente script se ha empleado el método **addBands()**, introducciéndole a este método el calculo de la diferencia normalizada. Esto lo que hará será incluir una nueva banda a cada una de las imágenes de la colección procesadas. Además, mediante el método **rename()** le estamos indicando que queremos denominar a esa banda con el nombre concreto NDVI.


In [94]:
def NDVI(entrada):
  ndvi = entrada.addBands(entrada.normalizedDifference(['B8','B4']).rename('NDVI'))
  return (ndvi)

coleccion_sentinel = ee.ImageCollection('COPERNICUS/S2_SR')\
           .filterBounds(parcela)\
           .filterDate('2018-01-01','2018-06-30')\
           .filterMetadata('CLOUD_COVERAGE_ASSESSMENT','less_than',5)

coleccion_sentinel2=coleccion_sentinel.map(NDVI)

print ()
print('Nombre de las bandas de la colección inicial')
imagen = ee.Image(coleccion_sentinel.first());
nombre_bandas=imagen.bandNames()
print (nombre_bandas.getInfo())

print ()
print('Nombre de las bandas de la colección con la banda de NDVI')
imagen = ee.Image(coleccion_sentinel2.first());
nombre_bandas=imagen.bandNames()
print (nombre_bandas.getInfo())


Nombre de las bandas de la colección inicial
['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B', 'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60']

Nombre de las bandas de la colección con la banda de NDVI
['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B', 'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60', 'NDVI']


### **5.1 Función para enmascarar nubes en una colección de imágenes Sentinel 2**

Se presenta la función **ESAcloudMask** para enmascarar la presencia de nubes en imaágenes Sentinel 2.


In [95]:
def ESAcloudMask(img):
    """
    European Space Agency (ESA) clouds from 'QA60', i.e. Quality Assessment band at 60m
    """
    qa = img.select('QA60')
    # bits 10 and 11 son nubes y cirros 
    cloudBitMask = int(2**10)
    cirrusBitMask = int(2**11)
    # Si 0 indicates condiciones claras.
    clear = qa.bitwiseAnd(cloudBitMask).eq(0).And(\
           qa.bitwiseAnd(cirrusBitMask).eq(0))
    
    # Nubes no estan claras
    cloud = clear.Not().rename(['ESA_clouds'])

    # Devuelve la banda de máscara de nubes.
    #return img.addBands(cloud)
    return img.updateMask(clear)

En el siguiente script se aplica la función de máscara de nubes de imágenes Sentinel 2, pintando en el visor la colección original y la enmascarada.

In [97]:
#Leemos un shapeFile alojado en nuetro espacio en Google Drive
path_file = os.path.join('/content/drive/MyDrive/Colab Notebooks/HIBA/Cuadernos Colab/3 - Colecciones de imagenes/datos/parcela_demo.shp')
parcela = geemap.shp_to_ee(path_file)

coleccion_sentinel = ee.ImageCollection('COPERNICUS/S2_SR')\
           .filterBounds(parcela)\
           .filterDate('2018-03-20','2018-03-22')

coleccion_sentinel_sin_nubes=coleccion_sentinel.map(ESAcloudMask)


vis_params = {
  'min': 0,
  'max': 5000,
  'bands': ['B8', 'B3', 'B4'],
}

poligono=ee.Geometry.Polygon(parcela.geometry().getInfo()['coordinates'])
centroide = poligono.centroid().getInfo()['coordinates']
centroide.reverse()

dicc = {
     'Imágenes originales' : coleccion_sentinel.getMapId(vis_params),
     'Máscara de nubes':coleccion_sentinel_sin_nubes.getMapId(vis_params),
     'parcelas':parcela.getInfo()
}

# Display the results
Mapdisplay(centroide, dicc, zoom_start= 10)
